## Решение


In [2]:
import pandas as pd


In [3]:
df1 = pd.read_csv('datasets/ferma_main.csv')
df2 = pd.read_csv('datasets/ferma_dad.csv')
df3 = pd.read_csv('datasets/cow_buy.csv')

In [4]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 634 entries, 0 to 633
Data columns (total 5 columns):
 #   Column                                                                                 Non-Null Count  Dtype 
---  ------                                                                                 --------------  ----- 
 0   id;Удой                                                                                634 non-null    object
 1    кг;ЭКЕ (Энергетическая кормовая единица);Сырой протеин                                634 non-null    object
 2    г;СПО (Сахаро-протеиновое соотношение);Порода;Тип пастбища;порода папы_быка;Жирность  634 non-null    object
 3   %;Белок                                                                                634 non-null    object
 4   %;Вкус молока;Возраст                                                                  616 non-null    object
dtypes: object(5)
memory usage: 24.9+ KB
